<a href="https://colab.research.google.com/github/ethvedbitdesjan/vedaant-jain-sectioning-info-extraction/blob/main/Text-Based-IE/EducationIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [ ]:
%cd /content/content/MyDrive/Resume_Info_Extraction/Education

/content/content/MyDrive/Resume_Info_Extraction/Education


In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.7 MB/s eta 0:00

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

from tqdm import tqdm
import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
def load_sentences(filepath):

    final = []
    sentences = []

    with open(filepath, 'r') as f:
        
        for line in f.readlines():
            
            if (line == ('-DOCSTART- -X- O\n') or line == '\n'):
                if len(sentences) > 0:
                    final.append(sentences)
                    sentences = []
            else:
                l = line.split(' ')
                sentences.append((l[0], l[3].strip('\n')))
    
    return final

In [ ]:
base_path = 'largest_data.conll'

train_samples = load_sentences(base_path)

samples = train_samples

schema = sorted({tag for sentence in samples 
                             for _, tag in sentence})

In [ ]:
schema_id = {label:i for i, label in enumerate(schema)}
schema_id

{'B-Degree': 0,
 'B-Thesis': 1,
 'B-University': 2,
 'I-Degree': 3,
 'I-Thesis': 4,
 'I-University': 5,
 'O': 6}

In [ ]:
samples = [[np.array([samp[0], schema_id[samp[1]]]) for samp in sample] for sample in samples]

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
samples= [np.array(sample) for sample in samples]
sample = samples[0]
samples_np = np.array(samples)
samples_np[0][:, 0]

<ipython-input-13-af730befd073>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  samples_np = np.array(samples)


array(['EDUCATION', 'Jan', ':', '1993', 'Dec.', '2003', 'PhD.',
       'Computer', 'Science', 'UNIVERSITY', 'OF', 'ILLINOIS', 'AT',
       'URBANA-CHAMPAIGN', 'Advisor', ':', 'Sam', 'Kamin', ';',
       'Dissertarion', 'citle', ':', 'Controlling', 'Self-Applicable',
       'Online', 'Partial', 'Evaluation', 'Strategies', '1989', 'Dee',
       '1992', 'B.S.', 'in', 'Mathemarics', 'and', 'Compucer', 'Science',
       'UNIVERSITY', 'OF', 'ILLINOIS', 'AT', 'URBANA-CHAMPAIGN'],
      dtype='<U21')

In [ ]:
custom_dataset = [{"tokens":sample[:, 0], "tags":sample[:, 1].astype(int)} for sample in samples]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    word_tokens = [token for token in examples['tokens']]
    tokenized_inputs = tokenizer(
        word_tokens, truncation=True, is_split_into_words=True)
    
    all_labels = examples["tags"]
    new_labels = []
    word_ids = tokenized_inputs.word_ids()
    new_labels = align_labels_with_tokens(all_labels, word_ids)

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
samples[0][:, 1]

array(['6', '6', '6', '6', '6', '6', '0', '3', '3', '2', '5', '5', '5',
       '5', '6', '6', '6', '6', '6', '6', '1', '4', '4', '4', '4', '4',
       '4', '4', '6', '6', '6', '0', '3', '3', '3', '3', '3', '2', '5',
       '5', '5', '5'], dtype='<U21')

In [ ]:
tokenized_dataset = [tokenize_and_align_labels(data) for data in custom_dataset]

In [ ]:
len(tokenized_dataset), tokenized_dataset[0]

(29,
 {'input_ids': [101, 142, 2137, 21986, 13821, 24805, 4945, 131, 1949, 13063, 119, 1581, 7735, 119, 6701, 2444, 7414, 26140, 8900, 12150, 3663, 11345, 15393, 2162, 11607, 2346, 6258, 13020, 158, 22672, 14962, 1592, 118, 24890, 10964, 10147, 23413, 2249, 20567, 131, 2687, 14812, 7937, 132, 12120, 14607, 6817, 1988, 172, 2875, 1513, 131, 6342, 1979, 9434, 118, 138, 8661, 9538, 2165, 10523, 4539, 2916, 27845, 1457, 5498, 19310, 2056, 9115, 1924, 139, 119, 156, 119, 1107, 15112, 14494, 4907, 1116, 1105, 3291, 8223, 15776, 1197, 2444, 7414, 26140, 8900, 12150, 3663, 11345, 15393, 2162, 11607, 2346, 6258, 13020, 158, 22672, 14962, 1592, 118, 24890, 10964, 10147, 23413, 2249, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_dataset[i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            0,    0,    3,    3,    2,    2,    2,    2,    2,    5,    5,    6,
            6,    6,    6,    5,    5,    6,    6,    6,    6,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            1,    2,    2,    4,    4,    4,    4,    4,    4,    4,    4,    4,
            4,    4,    4,    4,    4,    4,    4,    6,    6,    6,    0,    0,
            0,    0,    3,    3,    4,    4,    4,    3,    3,    4,    4,    4,
            3,    2,    2,    2,    2,    2,    5,    5,    6,    6,    6,    6,
            5,    5,    6,    6,    6,    6,    6,    6,    6,    6,    6, -100],
        [-100,    6,    0,    3,    4,    3,    3,    3,    3,    6,    2,    5,
            5,    5,    5,    6,    6,    6,    6,    6,    6,    6,    6,    6,
            0,    0,    0,    0,    3,    3,    3,    3,    3,    6,    2,    5,
            5,    5,    5, 

In [ ]:
for i in range(2):
    print(tokenized_dataset[i]["labels"])

[-100, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 3, 3, 2, 2, 2, 2, 2, 5, 5, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 0, 0, 0, 0, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 2, 2, 2, 2, 2, 5, 5, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100]
[-100, 6, 0, 3, 4, 3, 3, 3, 3, 6, 2, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 3, 3, 3, 3, 3, 6, 2, 5, 5, 5, 5, 6, 6, 6, 0, 0, 0, 0, 3, 3, 6, 2, 2, 5, 5, 6, 6, 6, 6, 6, 0, 0, 0, 3, 3, 6, 2, 2, 2, 5, 6, 6, 6, 6, 6, -100]


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6ef6358581aa61095f875a2f7c89e7fea43b967cca0ce0a7105d8fe0eb230a8c
  Stored in directory: /root/.cache/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
label_names = list(schema_id.keys())
label_names

['B-Degree',
 'B-Thesis',
 'B-University',
 'I-Degree',
 'I-Thesis',
 'I-University',
 'O']

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for label, i in schema_id.items()}
id2label

{0: 'B-Degree',
 1: 'B-Thesis',
 2: 'B-University',
 3: 'I-Degree',
 4: 'I-Thesis',
 5: 'I-University',
 6: 'O'}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=schema_id,
)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-c

In [ ]:
model.config.num_labels

7

In [ ]:
%mkdir bert_seq_tag
%cd /content/bert_seq_tag

mkdir: cannot create directory ‘bert_seq_tag’: File exists
[Errno 2] No such file or directory: '/content/bert_seq_tag'
/content/content/MyDrive/Resume_Info_Extraction/Education


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset[:10]+tokenized_dataset[15:],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=1,
)
eval_dataloader = DataLoader(
    tokenized_dataset[10:15], collate_fn=data_collator, batch_size=1
)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "bert_seq_tag"

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_labels, true_predictions = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        

  0%|          | 0/120 [00:00<?, ?it/s]

epoch 0: {'precision': 0.3076923076923077, 'recall': 0.3835616438356164, 'f1': 0.34146341463414637, 'accuracy': 0.7249417249417249}
epoch 1: {'precision': 0.6746987951807228, 'recall': 0.7671232876712328, 'f1': 0.717948717948718, 'accuracy': 0.8181818181818182}
epoch 2: {'precision': 0.7439024390243902, 'recall': 0.8356164383561644, 'f1': 0.7870967741935484, 'accuracy': 0.8344988344988346}
epoch 3: {'precision': 0.7228915662650602, 'recall': 0.821917808219178, 'f1': 0.7692307692307693, 'accuracy': 0.8438228438228438}
epoch 4: {'precision': 0.7439024390243902, 'recall': 0.8356164383561644, 'f1': 0.7870967741935484, 'accuracy': 0.8461538461538461}


In [ ]:
def map_token(a):
  if a not in id2label:
    return 'missing'
  return id2label[a]
list1 = [map_token(inp) for inp in predictions[0].cpu().detach().numpy()]
list2 = [map_token(inp) for inp in labels[0].cpu().detach().numpy()]

In [ ]:
list3 = []
for i in range(len(list1)):
  if list1[i] != list2[i]:
    list3.append(i)

In [ ]:
batch = data_collator([tokenized_dataset[3]])
batch["labels"]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch.to(device)
outputs = model(**batch)

In [ ]:
predictions = outputs.logits.argmax(dim=-1)
labels = batch["labels"]

In [ ]:
true_labels, true_predictions = postprocess(predictions, labels)

In [ ]:
schema_id['missing'] = 7

In [ ]:
schema_id

{'B-Degree': 0,
 'B-Thesis': 1,
 'B-University': 2,
 'I-Degree': 3,
 'I-Thesis': 4,
 'I-University': 5,
 'O': 6,
 'missing': 7}

In [ ]:
list1 = [map_token(inp) for inp in predictions[0].cpu().detach().numpy()]
list2 = [map_token(inp) for inp in labels[0].cpu().detach().numpy()]
pred_idx = [i for i in range(len(list1)) if schema_id[list1[i]] ==2 or schema_id[list1[i]]==5]
label_idx = [i for i in range(len(list2)) if schema_id[list2[i]] ==2 or schema_id[list2[i]]==5]
pred_tokens = [batch['input_ids'][0][i] for i in pred_idx]
batch['token_ids'] = pred_tokens
print(tokenizer.decode(**batch))
label_tokens = [batch['input_ids'][0][i] for i in label_idx]
batch['token_ids'] = label_tokens
#print('labels:\n', tokenizer.decode(**batch))

University of Florida, Gville, FL, USAsis University of Oregon, Eugene, OR, USAsis University of Maryland, Park, MD, USA University of Maryland, College Park, MD, USA


In [ ]:
batch['token_ids'] = batch['input_ids'][0]
print(tokenizer.decode(**batch))

[CLS] Education PH. D. in Computer Science, University of Florida, Gainesville, FL, USA ; 2016 ; Thesis : Designing and Leveraging Trustworthy Provenance - Aware Architectures Advisor : Dr. Kevin Butler M. S. in Computer Science, University of Oregon, Eugene, OR, USA ; 2012 ; Thesis : Detecting Compute Cloud Co - residency with Network Flow Watermarking Techniques ; Advisor : Dr. Kevin Butler : B. SC. in Computer Science, University of Maryland, Park, MD, USA ; December 2006. B. A. in English Literature, University of Maryland, College Park, MD, USA ; December 2006. [SEP]
